# Geopandas exploration

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip list| grep satellite

satellitecrops             0.0.1          /home/adbla/code/adblanq/satellite-crops


In [11]:
from satellitecrops.params import *

ModuleNotFoundError: No module named 'satellitecrops'

In [4]:
import sqlalchemy
import pg8000
from google.cloud.sql.connector import Connector, IPTypes

In [5]:
INSTANCE_NAME = "satellite-crops:europe-west9:satellitecrops1"
USER = "postgres"
SECRET = "x<v,e]'LTMn>T2^_"

In [6]:
def connect_with_connector() -> sqlalchemy.engine.base.Engine:
    """
    Initializes a connection pool for a Cloud SQL instance of Postgres.

    Uses the Cloud SQL Python Connector package.
    """
    # Note: Saving credentials in environment variables is convenient, but not
    # secure - consider a more secure solution such as
    # Cloud Secret Manager (https://cloud.google.com/secret-manager) to help
    # keep secrets safe.

    instance_connection_name = INSTANCE_NAME  # e.g. 'project:region:instance'
    db_user = USER  # e.g. 'my-db-user'
    db_pass = SECRET  # e.g. 'my-db-password'
    db_name = USER  # e.g. 'my-database'

    ip_type = IPTypes.PUBLIC
    # ip_type = IPTypes.PRIVATE if os.environ.get("PRIVATE_IP") else IPTypes.PUBLIC

    # initialize Cloud SQL Python Connector object
    connector = Connector()

    def getconn() -> pg8000.dbapi.Connection:
        conn: pg8000.dbapi.Connection = connector.connect(
            instance_connection_name,
            "pg8000",
            user=db_user,
            password=db_pass,
            db=db_name,
            ip_type=ip_type,
        )
        return conn

    # The Cloud SQL Python Connector can be used with SQLAlchemy
    # using the 'creator' argument to 'create_engine'
    pool = sqlalchemy.create_engine(
        "postgresql+pg8000://",
        creator=getconn,
        # ...
    )
    return pool


In [7]:
pool = connect_with_connector()

In [8]:
with pool.connect() as db_conn:
    select = sqlalchemy.text("""SELECT
    ST_ASGEOJSON(ST_Transform(geom,
        4326))
    FROM
    parcelles_graphiques
    LIMIT
    1""")
    result = db_conn.execute(select).fetchall()
    result

In [9]:
result[0][0]

'{"type":"Polygon","coordinates":[[[3.486003825,48.850872817],[3.479749458,48.850600206],[3.479819776,48.850796948],[3.479857503,48.8510937],[3.479894017,48.851647856],[3.485093595,48.851906285],[3.486003825,48.850872817]]]}'